# AMPL - solve multiple models in parallel
[![multiproc.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/multiprocessing/multiproc.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/colab.ampl.com/blob/master/authors/nfbvs/multiprocessing/multiproc.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/multiprocessing/multiproc.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/colab.ampl.com/blob/master/authors/nfbvs/multiprocessing/multiproc.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/colab.ampl.com/blob/master/authors/nfbvs/multiprocessing/multiproc.ipynb) [![Hits](https://h.ampl.com/https://github.com/ampl/colab.ampl.com/blob/master/authors/nfbvs/multiprocessing/multiproc.ipynb)](https://colab.ampl.com)

Description: Solve multiple AMPL models in parallel in Python with amplpy and the multiprocessing modules.

Tags: AMPL, Python, amplpy, multiprocess, Parallel Computing, Stochastic Programming

Notebook author: Nicolau Santos <<nicolau@ampl.com>>

In [31]:
# Install dependencies
%pip install -q amplpy

In [32]:
# Google Colab & Kaggle integration
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=["highs"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

Using default Community Edition License for Colab. Get yours at: https://ampl.com/ce
Licensed to AMPL Community Edition License for the AMPL Model Colaboratory (https://ampl.com/colab).


In [33]:
%%writefile model.mod

# Modelo de optimización para la producción de guantes industriales

# CONJUNTOS
set TRABAJOS; # Trabajos o lotes de producción
set MAQUINAS ordered; # Máquinas disponibles en el proceso (debe ser ordenado para usar first y last)
set FASES ordered; # Fases del proceso (debe ser ordenado para usar first y last) 1-troquelado, 2-selección, 3-armado, 4-cerrado, 5-volteado, 6-empacado
set MAQTRAB{i in TRABAJOS} within MAQUINAS ordered; # Máquinas asignadas para cada trabajo
set MAQFASES{f in FASES} within MAQUINAS ordered; # Máquinas disponibles para cada fase
set FASESTRAB{i in TRABAJOS} within FASES ordered; # Fases asignadas a cada trabajo

# PARÁMETROS
param t{i in TRABAJOS, f in FASES, j in MAQUINAS} >= 0 default 0; # Tiempo de procesamiento del trabajo i en la fase f con la máquina j
param d{i in TRABAJOS} >= 0; # Fecha de entrega del trabajo i
param H >= 0; # Número muy grande para restricciones

# VARIABLES
var C{i in TRABAJOS, f in FASES, j in MAQUINAS} >= 0; # Tiempo de culminación del trabajo i en la fase f con la máquina j
var S{i in TRABAJOS, f in FASES, j in MAQUINAS} >= 0; # Tiempo de inicio del trabajo i en la fase f con la máquina j
var W{j in MAQUINAS, i in TRABAJOS, k in TRABAJOS} >= 0; # Tiempo de espera del trabajo i en la máquina j
var T{i in TRABAJOS} >= 0; # Tardanza del trabajo i
var E{i in TRABAJOS} >= 0; # Adelanto del trabajo i
var X{i in TRABAJOS, k in TRABAJOS} binary; # Variable binaria para secuenciación de trabajos
var Y {j in MAQUINAS, q in MAQUINAS, i in TRABAJOS, k in TRABAJOS: j<>q} binary;# Variable binaria para secuenciación entre máquinas


# FUNCIÓN OBJETIVO
minimize tardanza_total:
    sum{i in TRABAJOS} (E[i] + T[i]) + sum{j in MAQUINAS, i in TRABAJOS, k in TRABAJOS} W[j, i, k];

# RESTRICCIONES
subject to precedencia{i in TRABAJOS, f in FASESTRAB[i]: ord(f) > 1}:
    C[i, f, first(MAQFASES[f])] >= C[i, prev(f), first(MAQFASES[prev(f)])] + t[i, prev(f), first(MAQFASES[prev(f)])];

subject to restriccion{j in MAQUINAS, i in TRABAJOS, k in TRABAJOS: i <> k}:
    S[k, first(FASESTRAB[k]), j] - (S[i, first(FASESTRAB[i]), j] + t[i, first(FASESTRAB[i]), j] + W[j, i, k]) + H * (1 - X[i, k]) >= 0;

subject to maquina{j in MAQUINAS, i in TRABAJOS, k in TRABAJOS: i <> k}:
    S[i, first(FASESTRAB[i]), j] - (S[k, first(FASESTRAB[k]), j] + t[k, first(FASESTRAB[k]), j] + W[j, k, i]) + H * X[i, k] >= 0;

subject to calculo{i in TRABAJOS}:
    T[i] - E[i] = C[i, last(FASESTRAB[i]), first(MAQFASES[last(FASESTRAB[i])])] - d[i];

subject to determinar{i in TRABAJOS, f in FASESTRAB[i], j in MAQFASES[f]}:
    C[i, f, j] = S[i, f, j] + t[i, f, j];

subject to restriccion1 {i in TRABAJOS, k in TRABAJOS, j in MAQUINAS, q in MAQUINAS: j <> q}:
    S[k, first(FASESTRAB[k]), j] - S[i, first(FASESTRAB[i]), q] + H * Y[j, q, i, k] >= 1;

subject to maquina1 {i in TRABAJOS, k in TRABAJOS, j in MAQUINAS, q in MAQUINAS: j <> q}:
     S[i, first(FASESTRAB[i]), q] - S[k, first(FASESTRAB[k]), j] + H * (1 - Y[j, q, i, k]) >= 1;

Overwriting model.mod


In [34]:
%%writefile datos.dat
#conjutos

set TRABAJOS := 1 2 3 4 5;

set MAQUINAS := 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28
    29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50;

set FASES := 1 2 3 4 5 6;

#maquinas que son usadas en cada fase

set MAQFASES[1] := 1 2 3 4;
set MAQFASES[2] := 5 6;
set MAQFASES[3] := 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32;
set MAQFASES[4] := 33 34 35 36 37 38 39 40 41;
set MAQFASES[5] := 42 43 44 45 46 47 48 49;
set MAQFASES[6] := 50;

#fases o procesos presentes en el trabajo

set FASESTRAB[1] := 1 2 3 4 5 6;
set FASESTRAB[2] := 1 2 3 4 5 6;
set FASESTRAB[3] := 1 2 3 4 5 6;
set FASESTRAB[4] := 1 2 3 4 5 6;
set FASESTRAB[5] := 1 2 3 4 5 6;

#tiempo de procesamiento del trabajo en las maquinas por cada pedido

# Parámetro de tiempos de procesamiento
param t :=
    # Fase 1: Máquinas 1, 2, 3, 4
    1 1 1 0.333
    1 1 2 0.333
    1 1 3 0.333
    1 1 4 0.333

    2 1 1 0.333
    2 1 2 0.333
    2 1 3 0.333
    2 1 4 0.333

    3 1 1 0.333
    3 1 2 0.333
    3 1 3 0.333
    3 1 4 0.333

    4 1 1 0.333
    4 1 2 0.333
    4 1 3 0.333
    4 1 4 0.333

    5 1 1 0.333
    5 1 2 0.333
    5 1 3 0.333
    5 1 4 0.333

    # Fase 2: Máquinas 5, 6
    1 2 5 0.083
    1 2 6 0.083

    2 2 5 0.083
    2 2 6 0.083

    3 2 5 0.083
    3 2 6 0.083

    4 2 5 0.083
    4 2 6 0.083

    5 2 5 0.083
    5 2 6 0.083

    # Fase 3: Máquinas 7 a 32
    1 3 7 0.666
    1 3 8 0.666
    1 3 9 0.666
    1 3 10 0.666
    1 3 11 0.666
    1 3 12 0.666
    1 3 13 0.666
    1 3 14 0.666
    1 3 15 0.666
    1 3 16 0.666
    1 3 17 0.666
    1 3 18 0.666
    1 3 19 0.666
    1 3 20 0.666
    1 3 21 0.666
    1 3 22 0.666
    1 3 23 0.666
    1 3 24 0.666
    1 3 25 0.666
    1 3 26 0.666
    1 3 27 0.666
    1 3 28 0.666
    1 3 29 0.666
    1 3 30 0.666
    1 3 31 0.666
    1 3 32 0.666

    2 3 7 0.666
    2 3 8 0.666
    2 3 9 0.666
    2 3 10 0.666
    2 3 11 0.666
    2 3 12 0.666
    2 3 13 0.666
    2 3 14 0.666
    2 3 15 0.666
    2 3 16 0.666
    2 3 17 0.666
    2 3 18 0.666
    2 3 19 0.666
    2 3 20 0.666
    2 3 21 0.666
    2 3 22 0.666
    2 3 23 0.666
    2 3 24 0.666
    2 3 25 0.666
    2 3 26 0.666
    2 3 27 0.666
    2 3 28 0.666
    2 3 29 0.666
    2 3 30 0.666
    2 3 31 0.666
    2 3 32 0.666

    3 3 7 0.666
    3 3 8 0.666
    3 3 9 0.666
    3 3 10 0.666
    3 3 11 0.666
    3 3 12 0.666
    3 3 13 0.666
    3 3 14 0.666
    3 3 15 0.666
    3 3 16 0.666
    3 3 17 0.666
    3 3 18 0.666
    3 3 19 0.666
    3 3 20 0.666
    3 3 21 0.666
    3 3 22 0.666
    3 3 23 0.666
    3 3 24 0.666
    3 3 25 0.666
    3 3 26 0.666
    3 3 27 0.666
    3 3 28 0.666
    3 3 29 0.666
    3 3 30 0.666
    3 3 31 0.666
    3 3 32 0.666

    4 3 7 0.666
    4 3 8 0.666
    4 3 9 0.666
    4 3 10 0.666
    4 3 12 0.666
    4 3 13 0.666
    4 3 14 0.666
    4 3 15 0.666
    4 3 16 0.666
    4 3 17 0.666
    4 3 18 0.666
    4 3 19 0.666
    4 3 20 0.666
    4 3 21 0.666
    4 3 22 0.666
    4 3 23 0.666
    4 3 24 0.666
    4 3 25 0.666
    4 3 26 0.666
    4 3 27 0.666
    4 3 28 0.666
    4 3 29 0.666
    4 3 30 0.666
    4 3 31 0.666
    4 3 32 0.666

    5 3 7 0.666
    5 3 8 0.666
    5 3 9 0.666
    5 3 10 0.666
    5 3 11 0.666
    5 3 12 0.666
    5 3 13 0.666
    5 3 14 0.666
    5 3 15 0.666
    5 3 16 0.666
    5 3 17 0.666
    5 3 18 0.666
    5 3 19 0.666
    5 3 20 0.666
    5 3 21 0.666
    5 3 22 0.666
    5 3 23 0.666
    5 3 24 0.666
    5 3 25 0.666
    5 3 26 0.666
    5 3 27 0.666
    5 3 28 0.666
    5 3 29 0.666
    5 3 30 0.666
    5 3 31 0.666
    5 3 32 0.666

    # Fase 4: Máquinas 33 a 41
    1 4 33 0.633
    1 4 34 0.633
    1 4 35 0.633
    1 4 36 0.633
    1 4 37 0.633
    1 4 38 0.633
    1 4 39 0.633
    1 4 40 0.633
    1 4 41 0.633

    2 4 33 0.633
    2 4 34 0.633
    2 4 35 0.633
    2 4 36 0.633
    2 4 37 0.633
    2 4 38 0.633
    2 4 39 0.633
    2 4 40 0.633
    2 4 41 0.633

    3 4 33 0.633
    3 4 34 0.633
    3 4 35 0.633
    3 4 36 0.633
    3 4 37 0.633
    3 4 38 0.633
    3 4 39 0.633
    3 4 40 0.633
    3 4 41 0.633

    4 4 33 0.633
    4 4 34 0.633
    4 4 35 0.633
    4 4 36 0.633
    4 4 37 0.633
    4 4 38 0.633
    4 4 39 0.633
    4 4 40 0.633
    4 4 41 0.633

    5 4 33 0.633
    5 4 34 0.633
    5 4 35 0.633
    5 4 36 0.633
    5 4 37 0.633
    5 4 38 0.633
    5 4 39 0.633
    5 4 40 0.633
    5 4 41 0.633

    # Fase 5: Máquinas 42 a 49
    1 5 42 0.25
    1 5 43 0.25
    1 5 44 0.25
    1 5 45 0.25
    1 5 46 0.25
    1 5 47 0.25
    1 5 48 0.25
    1 5 49 0.25

    2 5 42 0.25
    2 5 43 0.25
    2 5 44 0.25
    2 5 45 0.25
    2 5 46 0.25
    2 5 47 0.25
    2 5 48 0.25
    2 5 49 0.25

    3 5 42 0.25
    3 5 43 0.25
    3 5 44 0.25
    3 5 45 0.25
    3 5 46 0.25
    3 5 47 0.25
    3 5 48 0.25
    3 5 49 0.25

    4 5 42 0.25
    4 5 43 0.25
    4 5 44 0.25
    4 5 45 0.25
    4 5 46 0.25
    4 5 47 0.25
    4 5 48 0.25
    4 5 49 0.25

    5 5 42 0.25
    5 5 43 0.25
    5 5 44 0.25
    5 5 45 0.25
    5 5 46 0.25
    5 5 47 0.25
    5 5 48 0.25
    5 5 49 0.25


    # Fase 6: Máquina 50
    1 6 50 0.041
    2 6 50 0.041
    3 6 50 0.041
    4 6 50 0.041
    5 6 50 0.041
;

param d:=
1  168
2  336
3  504
4  672
5  840;

#Número muy grande
param H:= 10000;


Overwriting datos.dat


In [35]:
%%ampl_eval

model model.mod;
data datos.dat;
option solver cplex;
display tardanza_total;

printf "\nde maquina1\n";
display maquina1;

printf "\nprecedencia\n";
display precedencia;

tardanza_total = 0


de maquina1
maquina1 [1,1,*,*]
:    1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19 :=
1    .   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2    0   .   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3    0   0   .   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
4    0   0   0   .   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
5    0   0   0   0   .   0   0   0   0   0   0   0   0   0   0   0   0   0   0
6    0   0   0   0   0   .   0   0   0   0   0   0   0   0   0   0   0   0   0
7    0   0   0   0   0   0   .   0   0   0   0   0   0   0   0   0   0   0   0
8    0   0   0   0   0   0   0   .   0   0   0   0   0   0   0   0   0   0   0
9    0   0   0   0   0   0   0   0   .   0   0   0   0   0   0   0   0   0   0
10   0   0   0   0   0   0   0   0   0   .   0   0   0   0   0   0   0   0   0
11   0   0   0   0   0   0   0   0   0   0   .   0   0   0   0   0   0   0  